In [151]:
from matplotlib import pyplot as plt
from runRecordSplitter import *
from scipy.io import wavfile
import pandas as pd
import numpy as np

In [2]:
meetingAudioFiles = ['z.wav', 'zoom2.wav', 'P1.wav', 'P2.wav']
beepFile = 'beep.wav'
meetingBeeps = detectAllBeeps(meetingAudioFiles, beepFile)

For z;
6 Beep sounds has been detected.
For zoom2;
30 Beep sounds has been detected.
For P1;
39 Beep sounds has been detected.
For P2;
38 Beep sounds has been detected.


In [12]:
sampleRate, _ = wavfile.read('beep.wav')

In [149]:
audioFile = 'P2'
beeps = meetingBeeps[audioFile]
meetingAudio_sr, meeting = readMeetingAudio(audioFile+'.wav')

In [6]:
trailsFile = 'C:\\cStorage\\Datasets\\WhiteBallExp\\trails\\25fps\\data\\summaries_2021-01-14 02-02-21.csv'
trails = pd.read_csv(trailsFile)

In [7]:
trails

,name,duration,length,frameCount,cornerCount,stopCount,speed,fps,width,height,radius,stopRadius,stopStroke
0,infinity,21.44,8948,536,45,0,16,25,1920,1080,10,17,2
1,infinity_slow,44.16,8948,1104,45,0,8,25,1920,1080,10,17,2
2,random1,33.08,9686,827,11,9,16,25,1920,1080,10,17,2
3,random1_slow,57.28,9686,1432,11,9,8,25,1920,1080,10,17,2
4,random4,46.60,11602,1165,20,18,16,25,1920,1080,10,17,2
5,random4_slow,75.60,11602,1890,20,18,8,25,1920,1080,10,17,2
6,random5,31.52,5527,788,20,18,16,25,1920,1080,10,17,2
7,random5_slow,45.36,5527,1134,20,18,8,25,1920,1080,10,17,2
8,horizontal,65.24,26244,1631,24,0,16,25,1920,1080,10,17,2
9,vertical,79.84,32292,1996,36,0,16,25,1920,1080,10,17,2


In [11]:
printBeeps(beeps)

	01:57 - 117.89 0.31 5198971 5174580 5223362
	02:35 - 155.51 0.94 6857771 6852803 6862740
	02:47 - 167.70 0.89 7395497 7381939 7409055
	03:08 - 188.63 0.92 8318523 8312880 8324167
	04:17 - 257.60 0.91 11359941 11347850 11372033
	04:49 - 289.72 0.94 12776610 12771886 12781334
	05:02 - 302.32 0.90 13332229 13320279 13344179
	06:10 - 370.34 0.94 16331907 16327025 16336789
	06:25 - 385.59 0.92 17004372 16992281 17016463
	07:23 - 443.38 0.93 19553242 19548364 19558121
	07:39 - 459.54 0.93 20265800 20253201 20278400
	08:45 - 525.32 0.93 23166727 23161650 23171804
	08:57 - 537.57 0.92 23706918 23694758 23719078
	10:17 - 617.97 0.92 27252351 27247773 27256929
	10:30 - 630.25 0.92 27794019 27782260 27805779
	11:17 - 677.42 0.93 29874387 29869287 29879487
	11:33 - 693.31 0.91 30575023 30564214 30585833
	12:47 - 767.74 0.93 33857410 33852259 33862561
	13:03 - 783.29 0.92 34542963 34529831 34556096
	13:25 - 805.32 0.93 35514738 35508793 35520683
	13:38 - 818.23 0.92 36084116 36072688 36095544
	14:

In [222]:
def getTrailIntervals(beeps, trails, sampleRate):
    intervals = []
    for i in range(len(beeps)-1):
        beginIndex, endIndex = beeps[i][-1], beeps[i+1][-2]
        duration = (endIndex - beginIndex)/sampleRate
        frameCount = duration*trails['fps'][0]
        rang = 0.7
        for j in range(len(trails['duration'])):
            if duration > trails['duration'][j] - rang and \
                duration < trails['duration'][j] + rang and \
                duration > trails['duration'][j]:
                 intervals.append((trails.iloc[j]['name'], duration, trails['duration'][j],  
                                  frameCount, trails['frameCount'][j],
                                  beginIndex, endIndex, beeps[i], beeps[i+1]))
                 break
    return intervals

In [223]:
intervals = getTrailIntervals(beeps, trails, sampleRate)
print(len(intervals)) 

17


In [232]:
columns = ['Name', 'RecordingDuration', 'TrailDuration', 'RecordingFrameCount',  'TrailFrameCount', 'beginIndex', 'endIndex', 'BeginBeep', 'endBeep']
df = pd.DataFrame(intervals, columns = columns).set_index('Name')
df['TimeGap'] = df['RecordingDuration'] - df['TrailDuration']
df['FrameGap'] = df['RecordingFrameCount'] - df['TrailFrameCount']
df['BeginFrame'] 
df

,RecordingDuration,TrailDuration,RecordingFrameCount,TrailFrameCount,beginIndex,endIndex,BeginBeep,endBeep,TimeGap,FrameGap
Name,,,,,,,,,,
random5,31.742698,31.52,793.567460,788,11372033,12771886,"(04:17, 257.59503401360547, 0.9091912616144404...","(04:49, 289.7190476190476, 0.9375058477443823,...",0.222698,5.567460
zigzag,67.638231,67.52,1690.955782,1688,13344179,16327025,"(05:02, 302.3181179138322, 0.9034810889033907,...","(06:10, 370.3380272108844, 0.9363373512148795,...",0.118231,2.955782
random1_slow,57.412721,57.28,1435.318027,1432,17016463,19548364,"(06:25, 385.58666666666664, 0.9207692650496563...","(07:23, 443.3841723356009, 0.9331645349860951,...",0.132721,3.318027
horizontal,65.379819,65.24,1634.495465,1631,20278400,23161650,"(07:39, 459.5419501133787, 0.9258399139570582,...","(08:45, 525.3226077097505, 0.9253304690050417,...",0.139819,3.495465
vertical,80.015760,79.84,2000.393991,1996,23719078,27247773,"(08:57, 537.5718367346939, 0.9194757116304036,...","(10:17, 617.9671428571429, 0.9217178888005757,...",0.175760,4.393991
random4,46.791565,46.60,1169.789116,1165,27805779,29869287,"(10:30, 630.2498639455782, 0.9249434111597749,...","(11:17, 677.4237414965986, 0.9285773846986288,...",0.191565,4.789116
vertical_part2_slow,74.068617,73.88,1851.715420,1847,30585833,33852259,"(11:33, 693.311179138322, 0.9092105966292553, ...","(12:47, 767.7417233560091, 0.9269736016439619,...",0.188617,4.715420
infinity,21.603107,21.44,540.077664,536,34556096,35508793,"(13:03, 783.2871428571428, 0.9158920360322199,...","(13:25, 805.3228571428572, 0.9307275599005379,...",0.163107,4.077664
zigzag_part1_slow,73.306644,73.20,1832.666100,1830,36095544,39328367,"(13:38, 818.2339229024943, 0.9244047868684969,...","(14:51, 891.9155555555556, 0.9219726768569457,...",0.106644,2.666100


In [230]:
trailInterval = df.loc['random5']

In [231]:
trailInterval

RecordingDuration                                                31.7427
TrailDuration                                                      31.52
RecordingFrameCount                                              793.567
TrailFrameCount                                                      788
beginIndex                                                      11372033
endIndex                                                        12771886
BeginBeep              (04:17, 257.59503401360547, 0.9091912616144404...
endBeep                (04:49, 289.7190476190476, 0.9375058477443823,...
Name: random5, dtype: object

In [ ]:
timeGap = trailInterval['RecordingDuration '] 